This time I'll be using the OpenSubtitles subtitles to look for chengyu. The subtitles were already segmented using Jieba with a dictionary of chengyu added to ensure they were segmented correctly.

In [16]:
import json
from multiprocessing import Pool
import pickle
import time

from opencc import OpenCC

In [4]:
with open('idioms.txt') as f:
    idioms = f.read().split('\n')

with open('tm_subs_segmented.pkl', 'rb') as f1, open('mm_subs_segmented.pkl', 'rb') as f2:
    tm_seg = pickle.load(f1)
    mm_seg = pickle.load(f2)

In [14]:
# just a list of idioms
idioms[:10]

['一毛不拔',
 '掩耳盜鈴',
 '一丘之貉',
 '囫圇吞棗',
 '班門弄斧',
 '朝三暮四',
 '水落石出',
 '名落孫山',
 '胸有成竹',
 '竭澤而漁']

In [12]:
# these are supposed to be aligned, but not all of them are perfect...
for t, m in islice(zip(tm_seg, mm_seg), 10):
    print(" ".join(t))
    print(" ".join(m))
    print()

老兄 她 少元
我们 为 什么 不 邀 请 呢

我 討厭 美式足球 說 到 我 的 運動 我 可是 十賭 九贏
算了 将 身家性命 都 押 上

他 到底 是 個 什麼 德行
他 竟敢 在 报上 说 我 觉得 我 爸 让 我 丢脸

妳 太 客氣 了
你 真 好

再來點 香檳
再来 一点 香槟 吗

殺 的 凶手 如果 可以 的 話
刺杀 林肯 的 凶手

起立 謝謝
请 起立 谢谢 先生

我 的 腳
救命 啊

你好 謝謝 你 的 邀 請
嗨 感谢 邀请 我们

便會 趁 虛而入
他们 就 会 发起 攻击



Below is my original code. I used our server to run it since it would be much faster running in parallel. It still took ~24h to complete though!

In [ ]:
def chengyu_counter(idiom):
    s2tw = OpenCC('s2tw')
    dic = {}

    temp = {'tm': [], 'mm': [], 'both': []}
    for idx, (t, m) in enumerate(zip(tm_seg, mm_seg)):
        m = [s2tw.convert(char) for char in m]
        if idiom in t:
            temp['tm'].append(idx)
        if idiom in m:
            temp['mm'].append(idx)
        if idiom in t and idiom in m:
            temp['both'].append(idx)
    dic[idiom] = temp
    return dic


if __name__ == '__main__':

    print('Starting...')
    start = time.time()

    with open('tm_subs_segmented.pkl', 'rb') as f1, open('mm_subs_segmented.pkl', 'rb') as f2:
        tm_seg = pickle.load(f1)
        mm_seg = pickle.load(f2)

    with open('idioms.txt') as f:
        idioms = f.read().split('\n')

    pool_size = 26

    with Pool(processes=pool_size) as pool:

        results = pool.map(chengyu_counter, idioms)

    with open('chengyu_freq_results.pkl', 'wb') as f:
        pickle.dump(results, f)

    stop = time.time()

    elasped = (stop - start) / 60
    print(f'Elapsed time: {elapsed}')

In [22]:
# Results, after combining all results into a dictionary
with open('chengyu_freq.json') as f:
    chengyu_dict = json.load(f)

In [23]:
chengyu_dict.keys()

dict_keys(['一毛不拔', '掩耳盜鈴', '一丘之貉', '囫圇吞棗', '班門弄斧', '朝三暮四', '水落石出', '名落孫山', '胸有成竹', '竭澤而漁', '指鹿為馬', '天衣無縫', '舉一反三', '江郎才盡', '青出於藍', '口若懸河', '驚弓之鳥', '含沙射影', '一敗塗地', '草木皆兵', '投筆從戎', '東窗事發', '畫蛇添足', '勢如破竹', '滄海桑田', '望梅止渴', '司空見慣', '南轅北轍', '夜郎自大', '信口雌黃', '狼狽為奸', '狐假虎威', '作法自斃', '為虎作倀', '如火如荼', '老馬識途', '杞人憂天', '一暴十寒', '守株待兔', '不學無術', '四面楚歌', '奇貨可居', '刻舟求劍', '走馬看花', '後來居上', '負荊請罪', '對牛彈琴', '一鼓作氣', '杯弓蛇影', '紙上談兵', '破鏡重圓', '東山再起', '樂不思蜀', '濫竽充數', '打草驚蛇', '揠苗助長', '功虧一簣', '入木三分', '一鳴驚人', '螳臂當車', '死灰復燃', '咄咄逼人', '出奇制勝', '出類拔萃', '一竅不通', '食古不化', '一日千里', '吹毛求疵', '一諾千金', '騎虎難下', '投鼠忌器', '孤注一擲', '坐井觀天', '滄海一粟', '門可羅雀', '攀龍附鳳', '痛心疾首', '易如反掌', '膾炙人口', '趾高氣揚', '唾面自乾', '推心置腹', '病入膏肓', '蕭規曹隨', '庸人自擾', '緣木求魚', '雙管齊下', '一言九鼎', '自相矛盾', '扶搖直上', '舉案齊眉', '閉門造車', '歧路亡羊', '風聲鶴唳', '老當益壯', '車水馬龍', '未雨綢繆', '先發制人', '洛陽紙貴', '孤陋寡聞', '不共戴天', '滿腹經綸', '飲鴆止渴', '退避三舍', '世外桃源', '剛愎自用', '出爾反爾', '目不識丁', '水深火熱', '假公濟私', '孤掌難鳴', '偃旗息鼓', '妄自尊大', '百折不撓', '拾人牙慧', '耳濡目染', '得心應手', '乘風破浪', '杯水車薪', '車載斗量', '滿城風雨', '燃眉之急', '言不由衷', '敝帚自珍

In [24]:
chengyu_dict

{'一毛不拔': {'tm': [1794156, 1990982],
  'mm': [594786, 1589921, 1756650, 1937478, 2135352],
  'both': [],
  'tm_count': 2,
  'mm_count': 5,
  'both_count': 7},
 '掩耳盜鈴': {'tm': [],
  'mm': [108800, 965629, 1157264, 2336918],
  'both': [],
  'tm_count': 0,
  'mm_count': 4,
  'both_count': 4},
 '一丘之貉': {'tm': [61565, 119688, 1583099],
  'mm': [38711, 249484, 752653, 1566925],
  'both': [],
  'tm_count': 3,
  'mm_count': 4,
  'both_count': 7},
 '囫圇吞棗': {'tm': [832268, 1059467, 1126130],
  'mm': [],
  'both': [],
  'tm_count': 3,
  'mm_count': 0,
  'both_count': 3},
 '班門弄斧': {'tm': [42487, 1038443, 1687957, 2099632, 2617419, 2690273],
  'mm': [288252,
   964460,
   1038443,
   2325356,
   2363344,
   2366337,
   2476043,
   2690273,
   2696188],
  'both': [1038443, 2690273],
  'tm_count': 6,
  'mm_count': 9,
  'both_count': 15},
 '朝三暮四': {'tm': [],
  'mm': [241353, 2678306],
  'both': [],
  'tm_count': 0,
  'mm_count': 2,
  'both_count': 2},
 '水落石出': {'tm': [103551,
   194837,
   199608,
   2

In [25]:
tm_chengyu = sorted(chengyu_dict.items(), key=lambda item: item[1].get('tm_count'), reverse=True)
mm_chengyu = sorted(chengyu_dict.items(), key=lambda item: item[1].get('mm_count'), reverse=True)

Below are chengyu sorted by frequency.

In [26]:
for t, m in zip(tm_chengyu, mm_chengyu):
    print(f'{t[0]} ({t[1]["tm_count"]}), {m[0]} ({m[1]["mm_count"]})')

不可思議 (780), 不可思議 (824)
一無所知 (316), 一無所知 (339)
無能為力 (315), 無能為力 (311)
一無所有 (242), 一無所有 (235)
自以為是 (224), 自以為是 (200)
迫不及待 (148), 袖手旁觀 (147)
隨心所欲 (148), 迫不及待 (120)
袖手旁觀 (136), 安然無恙 (115)
逍遙法外 (118), 隨心所欲 (106)
束手無策 (114), 走投無路 (100)
一文不值 (110), 胡思亂想 (98)
輕舉妄動 (108), 逍遙法外 (98)
夢寐以求 (101), 大驚小怪 (95)
走投無路 (95), 一文不值 (93)
大驚小怪 (94), 瞭如指掌 (87)
胡思亂想 (88), 輕而易舉 (85)
起死回生 (88), 無與倫比 (82)
自欺欺人 (86), 無所事事 (80)
無與倫比 (83), 夢寐以求 (78)
循規蹈矩 (83), 輕舉妄動 (77)
無所事事 (81), 喋喋不休 (76)
格殺勿論 (80), 情不自禁 (75)
安然無恙 (80), 毛骨悚然 (73)
罪魁禍首 (78), 守口如瓶 (71)
輕而易舉 (73), 罪魁禍首 (71)
情不自禁 (71), 形影不離 (71)
名副其實 (71), 循規蹈矩 (70)
來龍去脈 (70), 心不在焉 (67)
光天化日 (70), 烏合之眾 (66)
洗耳恭聽 (68), 束手無策 (65)
形影不離 (67), 坐以待斃 (64)
按部就班 (66), 起死回生 (64)
忘恩負義 (66), 行屍走肉 (64)
守口如瓶 (65), 天涯海角 (63)
喋喋不休 (64), 自欺欺人 (63)
隨機應變 (64), 不知所措 (62)
忠心耿耿 (64), 光天化日 (62)
坐以待斃 (63), 無可救藥 (61)
刮目相看 (62), 心煩意亂 (60)
拭目以待 (61), 隨機應變 (59)
行屍走肉 (61), 拭目以待 (59)
不速之客 (60), 無濟於事 (59)
無可救藥 (60), 孤注一擲 (58)
史無前例 (59), 不速之客 (57)
回心轉意 (58), 虛張聲勢 (57)
寡不敵眾 (58), 鬼鬼祟祟 (56)
分道揚鑣 (57), 

不屑一世 (0), 不同凡調 (0)
不同凡調 (0), 玩兵黷武 (0)
玩兵黷武 (0), 不屈撓 (0)
不屈撓 (0), 不展愁眉 (0)
不展愁眉 (0), 不留餘力 (0)
不留餘力 (0), 不息自強 (0)
不息自強 (0), 巧取強奪 (0)
巧取強奪 (0), 頑冥不靈 (0)
頑冥不靈 (0), 巧逾造化 (0)
巧逾造化 (0), 巧奪造化 (0)
巧奪造化 (0), 不阿不撓 (0)
不阿不撓 (0), 不同凡流 (0)
不同凡流 (0), 巧絕天工 (0)
巧絕天工 (0), 不同天地 (0)
不同天地 (0), 北轅適越 (0)
北轅適越 (0), 不墮箕裘 (0)
不墮箕裘 (0), 引玉拋磚 (0)
引玉拋磚 (0), 班門取罪 (0)
班門取罪 (0), 不暇暖席 (0)
不暇暖席 (0), 引經據禮 (0)
引經據禮 (0), 引經據義 (0)
引經據義 (0), 引盜入闥 (0)
引盜入闥 (0), 不探虎穴，安得虎子 (0)
不探虎穴，安得虎子 (0), 北轅南轍 (0)
北轅南轍 (0), 背馳於道 (0)
背馳於道 (0), 背道而行 (0)
背道而行 (0), 背本逐末 (0)
背本逐末 (0), 驥伏櫪 (0)
驥伏櫪 (0), 蜚流之言 (0)
蜚流之言 (0), 不脛而馳 (0)
不脛而馳 (0), 引虎入室 (0)
引虎入室 (0), 背邪向正 (0)
背邪向正 (0), 張膽明目 (0)
張膽明目 (0), 巧舌花脣 (0)
巧舌花脣 (0), 巧侔造化 (0)
巧侔造化 (0), 刊陳出新 (0)
刊陳出新 (0), 形槁心灰 (0)
形槁心灰 (0), 引賊入家 (0)
引賊入家 (0), 引人勝地 (0)
引人勝地 (0), 孤行一意 (0)
孤行一意 (0), 孤詣苦心 (0)
孤詣苦心 (0), 孤標峻節 (0)
孤標峻節 (0), 孤芳自愛 (0)
孤芳自愛 (0), 不暇應接 (0)
不暇應接 (0), 水乳之合 (0)
水乳之合 (0), 水乳相和 (0)
水乳相和 (0), 水息不通 (0)
水息不通 (0), 水闊天空 (0)
水闊天空 (0), 水淺長流 (0)
水淺長流 (0), 水清石出 (0)
水清石出 (0), 水涸石出 (0)
水涸石出 (0), 水深火烈 (0)
水深火烈 (